In [1]:
open Core

Findlib has been successfully loaded. Additional directives:
  #require "package";;      to load a package
  #list;;                   to list the available packages
  #camlp4o;;                to load camlp4 (standard syntax)
  #camlp4r;;                to load camlp4 (revised syntax)
  #predicates "p,q,...";;   to set these predicates
  Topfind.reset();;         to force that packages will be reloaded
  #thread;;                 to enable threads



/home/ceclinux/.opam/4.04.1/lib/ocaml/dynlink.cma: loaded
/home/ceclinux/.opam/4.04.1/lib/ocaml/camlp4: added to search path
/home/ceclinux/.opam/4.04.1/lib/ocaml/camlp4/camlp4o.cma: loaded
/home/ceclinux/.opam/4.04.1/lib/ocaml/threads: added to search path


Cannot find file /home/ceclinux/.opam/4.04.1/lib/ocaml/camlp4/camlp4o.cma.


/home/ceclinux/.opam/4.04.1/lib/ocaml/unix.cma: loaded
/home/ceclinux/.opam/4.04.1/lib/ocaml/threads/threads.cma: loaded
/home/ceclinux/.opam/4.04.1/lib/base/caml: added to search path
/home/ceclinux/.opam/4.04.1/lib/base/caml/caml.cma: loaded
/home/ceclinux/.opam/4.04.1/lib/base/shadow_stdlib: added to search path
/home/ceclinux/.opam/4.04.1/lib/base/shadow_stdlib/shadow_stdlib.cma: loaded
/home/ceclinux/.opam/4.04.1/lib/sexplib/0: added to search path
/home/ceclinux/.opam/4.04.1/lib/sexplib/0/sexplib0.cma: loaded
/home/ceclinux/.opam/4.04.1/lib/base: added to search path
/home/ceclinux/.opam/4.04.1/lib/base/base.cma: loaded
/home/ceclinux/.opam/4.04.1/lib/base/md5: added to search path
/home/ceclinux/.opam/4.04.1/lib/base/md5/md5_lib.cma: loaded
/home/ceclinux/.opam/4.04.1/lib/ocaml/bigarray.cma: loaded
/home/ceclinux/.opam/4.04.1/lib/fieldslib: added to search path
/home/ceclinux/.opam/4.04.1/lib/fieldslib/fieldslib.cma: loaded
/home/ceclinux/.opam/4.04.1/lib/ppx_compare/runtime-lib

Ocaml最棒的特性之一是，它提供了一个简洁而且表达能力很强的系统来声明新的数据类型，记录则是这个系统中一个关键的要素

需要说明，记录字段名必须以一个小写的子母开头

In [2]:
type host_info = 
{hostname: string;
os_name:string;
cpu_arch:string;
timestamp:Time.t;
}

type host_info = {
  hostname : string;
  os_name : string;
  cpu_arch : string;
  timestamp : Core.Time.t;
}


In [3]:
# require "core_extended" 

No such package: core_extended


In [4]:
open Core_extended


error: compile_error

In [5]:
let my_host = 
{
hostname = "hostname";
os_name = "uname -s";
cpu_arch = "uname -p";
timestamp = Time.now();
}

val my_host : host_info =
  {hostname = "hostname"; os_name = "uname -s"; cpu_arch = "uname -p";
   timestamp = 2018-01-30 08:24:20.673660-08:00}


你可能想知道编译器如何推断出`my_host`类型是`host_info`。在这里，编译器确定类型的关键是记录字段名。在这一张后面，我们还会讨论多个记录类型有相同的字段名时会发生什么

可以使用点击法来抽取元素

In [6]:
my_host.cpu_arch;;

- : string = "uname -p"


声明Ocam类型时，都可以利用多态类型来进行参数化，在这方面记录也不例外。

In [7]:
type 'a timestamped = {item: 'a; time: Time.t};;

type 'a timestamped = { item : 'a; time : Core.Time.t; }


In [8]:
let first_timestamped list =
List.reduce list ~f:(fun a b -> if a .time < b.time then a else b)

val first_timestamped : 'a timestamped Core.List.t -> 'a timestamped option =
  <fun>


In [9]:
let host_info_to_string {hostname = h; os_name = os; cpu_arch = c; timestamp = ts;}
= sprintf "%s (%s / %s, on %s)" h os c "wa";;

val host_info_to_string : host_info -> string = <fun>


In [10]:
host_info_to_string my_host

- : string = "hostname (uname -s / uname -p, on wa)"


需要说明的是，我们使用的模式只有一种情况，这里没有使用由|分割的多个情况。我们只要一个模式，因为记录模式是“不可否定的”，只标明运行时记录模式匹配绝对不会失败。这是有道理的，因为记录中的字段集总是一样的。一般来讲，对于有固定结构的类型（如记录和元祖），其模式都是不可否定的，这与有可变结构的类型不同。

举个例子，假设我们希望向`host_info`记录添加一个新字段，名为`os_release`

In [11]:
type host_info = {
hostname: string;
os_name: string;
cpu_arch :string;
os_release: string;
timestamp: Time.t;
}

type host_info = {
  hostname : string;
  os_name : string;
  cpu_arch : string;
  os_release : string;
  timestamp : Core.Time.t;
}


In [12]:
host_info_to_string
let host_info_to_string {hostname = h; os_name = os; cpu_arch = c; timestamp = ts;}
= sprintf "%s (%s / %s, on %s)" h os c "wa";;

- : host_info -> string = <fun>


val host_info_to_string : host_info -> string = <fun>


host_info_to_string的代码不做修改仍能编译。对于这个情况，很显然你可能希望`host_info_to_string`来包含`os_release`，如果类型系统能对这个修改提供一个警告就好了

In [13]:
#warnings "+9"

In [17]:
let host_info_to_string {hostname = h; os_name = os; cpu_arch = c; timestamp = ts;}
= sprintf "%s (%s / %s, on %s)" h os c "wa";;

File "[17]", line 1, characters 24-83:
Warning 9: the following labels are not bound in this record pattern:
os_release
Either bind these labels explicitly or add '; _' to the pattern.


val host_info_to_string : host_info -> string = <fun>


可以禁用对一个给定模式的警告，明确地确认我们打算忽略额外的字段。为此可以为模式增加一个下划线

In [16]:
host_info_to_string
let host_info_to_string {hostname = h; os_name = os; cpu_arch = c; timestamp = ts;_}
= sprintf "%s (%s / %s, on %s)" h os c "wa";;

- : host_info -> string = <fun>


val host_info_to_string : host_info -> string = <fun>
